# Amazon Bedrock Agent ~ The CRM Example
Before running this notebook, make sure you have the following:
- AWS CLI installed and configured with your credentials.
- Access to the Amazon Bedrock service and the configured foundational model.

The goal of this notebook is to demonstrate the basic usage of the Amazon Bedrock Inline Agent.

In [ ]:
import boto3
from dotenv import load_dotenv

from bedrock_agent.crm.marketing_agent import MarketingAgent

Load environment variables and set up the AWS client

In [ ]:
_ = load_dotenv()

# Register new client using AWS CLI Authentication and our default region
region = "eu-west-1"
bedrock_rt_client = boto3.client(
    "bedrock-agent-runtime",
    region_name=region
)


Initialize the Marketing Agent with defaults. You can pass a different foundational model and session ID if needed.

In [ ]:
marketing_agent = MarketingAgent()

Create the request parameters by adding the question to the input text. Call the inline agent with the request parameters.

In [ ]:
request_params = marketing_agent.prepare_input(input_text="What is the mission of your company?")

_agent_resp = bedrock_rt_client.invoke_inline_agent(
    **request_params
)

Handle the response from the agent. The response is in a stream format, so we need to decode it and print it.

In [ ]:
if _agent_resp["ResponseMetadata"]["HTTPStatusCode"] != 200:
    _error_message = f"API Response was not 200: {_agent_resp}"
    print(_error_message)
else:
    try:
        _event_stream = _agent_resp["completion"]
        _agent_answer = ""
        for _event in _event_stream:

            if "chunk" in _event:
                _data = _event["chunk"]["bytes"]
                _agent_answer = _data.decode("utf8")

        print(f"Agent response:\n{_agent_answer}")
    except Exception as e:
        print(f"Caught exception while processing input to invokeAgent:\n")
        print(f"Error: {e}")


The response contains a lot more information than just the answer.